## Install and Import Dependencies

Installs required packages and imports all libraries used below.


In [19]:
import json
import time
from pathlib import Path
from datetime import datetime

import nest_asyncio
import together
from collinear.client import Client
from together.abstract import api_requestor
from together.types import TogetherRequest
from jinja2 import Environment, FileSystemLoader
from pprint import pprint

# Necessary to run in a Jupyter notebook
nest_asyncio.apply()


## Utility functions

Defines helpers to build steering personas, format conversations, and enrich results.


In [63]:
from datetime import datetime
def header(title: str) -> None:
    line = "=" * len(title)
    print(line)
    print(title)
    print(line)

def _summarize_results(path: Path) -> None:
    header("Evaluation Results")
    with path.open('r', encoding='utf-8') as results_file:
        for idx, line_text in enumerate(results_file, start=1):
            line_text = line_text.strip()
            if not line_text:
                continue
            try:
                result_row = json.loads(line_text)
            except Exception:
                header(f"Evaluation {idx}")
                print(line_text)
                continue
            score = result_row.get('score')
            passed = result_row.get('pass')
            feedback = result_row.get('feedback') or result_row.get('rationale') or ''
            status = 'PASS' if isinstance(passed, bool) and passed else ('FAIL' if isinstance(passed, bool) else '-')
            header(f"Evaluation {idx}")
            print(f"Score: {score if score is not None else '-'}  Status: {status}")
            if feedback:
                print('Reason:')
                print(feedback)

            conversation = result_row.get('conversation')
            if isinstance(conversation, str) and conversation:
                print()
                print('Conversation:')
                print(conversation)

            assistant_response = result_row.get('assistant_response')
            if isinstance(assistant_response, str) and assistant_response:
                print()
                print('Assistant response:')
                print(assistant_response)

            persona = result_row.get('steering_persona') or {}
            characteristics = persona.get('characteristics') if isinstance(persona, dict) else None
            traits = persona.get('traits') if isinstance(persona, dict) else None
            if isinstance(characteristics, dict) and characteristics:
                print()
                print('Characteristics:')
                for key, value in characteristics.items():
                    print(f"  {key}: {value}")
            if isinstance(traits, dict) and traits:
                print()
                print('Traits:')
                for key, value in traits.items():
                    print(f"  {key}: {value}")
            print()
def build_steering_persona(simulation_runner, steer_combination):
    """Return persona metadata via the runner's normalization helpers."""
    if steer_combination is None:
        return {'characteristics': {}, 'traits': {}}
    characteristics: dict[str, object] = {}
    try:
        characteristics = simulation_runner._user_characteristics_payload(steer_combination)
    except Exception:
        characteristics = {}
    traits = getattr(steer_combination, 'traits', {}) or {}
    return {'characteristics': characteristics, 'traits': traits}

def format_conversation(conversation_prefix):
    """Compact conversation text from message dicts."""
    return '\n'.join(
        f"{message.get('role', '')}: {message.get('content', '')}"
        for message in conversation_prefix
        if message.get('content')
    )

def make_dataset_row(simulation_runner, simulation):
    """Build a single dataset row with persona characteristics included."""
    return {
        'conversation': format_conversation(simulation.conv_prefix),
        'last_response': simulation.response,
        'steering_persona': build_steering_persona(simulation_runner, getattr(simulation, 'steer', None)),
    }

def load_persona_lookup(dataset_path: Path):
    """Map (conversation, assistant_response) -> steering_persona from dataset."""
    persona_lookup = {}
    with dataset_path.open('r', encoding='utf-8') as dataset_file:
        for line_text in dataset_file:
            try:
                dataset_row = json.loads(line_text)
            except Exception:
                continue
            join_key = (dataset_row.get('conversation'), dataset_row.get('assistant_response'))
            steering_persona = dataset_row.get('steering_persona')
            if isinstance(join_key[0], str) and isinstance(join_key[1], str) and isinstance(steering_persona, dict):
                persona_lookup[join_key] = steering_persona
    return persona_lookup

def enrich_results_inplace(results_path: Path, persona_lookup):
    """Attach steering_persona to Together results in place using lookup."""
    updated_lines = []
    with results_path.open('r', encoding='utf-8') as results_file:
        for line_text in results_file:
            try:
                result_row = json.loads(line_text)
                steering_persona = persona_lookup.get((result_row.get('conversation'), result_row.get('assistant_response')))
                if isinstance(steering_persona, dict):
                    result_row['steering_persona'] = steering_persona
                updated_lines.append(json.dumps(result_row, ensure_ascii=False) + '\n')
            except Exception:
                updated_lines.append(line_text)
    with results_path.open('w', encoding='utf-8') as results_file:
        results_file.writelines(updated_lines)

def display_persona(sim_runner, simulation):
    raw = build_steering_persona(sim_runner, getattr(simulation, 'steer', None))
    persona = {
      'characteristics': dict(raw.get('characteristics', {})),
      'traits': dict(raw.get('traits', {})),
    }
    pprint(persona)

def _slugify_filename(value: str, default: str = 'value') -> str:
    allowed = set('abcdefghijklmnopqrstuvwxyz0123456789-_')
    text = (value or '').lower().replace(' ', '-')
    cleaned = ''.join(ch for ch in text if ch in allowed)
    return cleaned or default

def _format_numeric_for_filename(value) -> str:
    if isinstance(value, float):
        formatted = f"{value:.2f}".rstrip('0').rstrip('.')
    else:
        formatted = str(value)
    return formatted.replace('.', 'p')



## Load Config

Loads simulation, judge, and Together settings from JSON files.


In [64]:
# Config Variables (from simulation_config.json and steering_config_*.json)

CONFIG_DIR = Path('configs')
SIMULATION_CONFIG_FILE = CONFIG_DIR / 'simulation_config.json'
config_data = json.loads(SIMULATION_CONFIG_FILE.read_text())

steering_config_value = config_data.get('steering_config_file') or 'steering_config_airline.json'
steering_candidate = Path(steering_config_value)
if not steering_candidate.is_absolute():
    steering_candidate = CONFIG_DIR / steering_candidate.name
STEERING_CONFIG_FILE = steering_candidate
STEER_CONFIG = json.loads(STEERING_CONFIG_FILE.read_text())
STEER_TASKS = list(STEER_CONFIG.get('tasks', []))

# Client options
client_settings = config_data.get('client', {}) or {}
CLIENT_ASSISTANT_MODEL_URL = client_settings.get('assistant_model_url', 'https://api.together.xyz/v1')
CLIENT_ASSISTANT_MODEL_API_KEY = client_settings.get('assistant_model_api_key')
CLIENT_ASSISTANT_MODEL_NAME = client_settings.get('assistant_model_name', 'meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo')
CLIENT_STEER_API_KEY = client_settings.get('steer_api_key', 'demo-001')
CLIENT_TIMEOUT = int(client_settings.get('timeout', 120))
CLIENT_MAX_RETRIES = int(client_settings.get('max_retries', 3))
CLIENT_RATE_LIMIT_RETRIES = int(client_settings.get('rate_limit_retries', 6))

# Simulation options
simulate_settings = config_data.get('simulate', {}) or {}
SIM_SAMPLES = simulate_settings.get('k', 3)
SIM_EXCHANGES = simulate_settings.get('num_exchanges', 2)
SIM_DELAY = simulate_settings.get('batch_delay', 0.2)
SIM_STEER_TEMPERATURE = simulate_settings.get('steer_temperature', 0.7)
SIM_STEER_MAX_TOKENS = simulate_settings.get('steer_max_tokens', 256)
SIM_MIX_TRAITS = bool(simulate_settings.get('mix_traits', False))
SIM_MAX_CONCURRENCY = int(simulate_settings.get('max_concurrency', 8))

# Assessment options
assess_settings = config_data.get('assess', {}) or {}
ASSESS_JUDGE_MODEL_URL = assess_settings.get('judge_model_url')
ASSESS_JUDGE_MODEL_API_KEY = assess_settings.get('judge_model_api_key')
ASSESS_JUDGE_MODEL_NAME = assess_settings.get('judge_model_name')

# Together-specific options
together_settings = config_data.get('together', {}) or {}
SIMULATED_CONVERSATION_DIRECTORY = together_settings.get('output_directory', '.')
SIMULATED_CONVERSATION_FILENAME = together_settings.get('output_filename', 'collinear_eval_dataset')

judge_template_value = together_settings.get('judge_system_prompt') or 'configs/judge_system_prompt.jinja'
judge_template_path = Path(judge_template_value)
JUDGE_SYSTEM_PROMPT = judge_template_path.read_text(encoding='utf-8')

TOGETHER_UPLOAD_PURPOSE = together_settings.get('upload_purpose', 'eval')
TOGETHER_EVAL_TYPE = together_settings.get('evaluation_type', 'score')
TOGETHER_MODEL_TO_EVALUATE = together_settings.get('model_to_evaluate', 'assistant_response')
TOGETHER_JUDGE_MODEL_SOURCE = together_settings.get('judge_model_source', 'serverless')
TOGETHER_MIN_SCORE = together_settings.get('min_score', 1.0)
TOGETHER_MAX_SCORE = together_settings.get('max_score', 10.0)
TOGETHER_PASS_THRESHOLD = together_settings.get('pass_threshold', 7.0)
TOGETHER_POLL_TIMEOUT_SECONDS = int(together_settings.get('poll_timeout_seconds', 300))
TOGETHER_POLL_INTERVAL_SECONDS = int(together_settings.get('poll_interval_seconds', 5))
RESULTS_FILENAME_PREFIX = together_settings.get('results_filename_prefix', 'together_eval_')

# Derived output metadata
RESULTS_DIR = Path(SIMULATED_CONVERSATION_DIRECTORY) / 'results'
TRAIT_CONFIG = STEER_CONFIG.get('traits') or {}
TRAIT_KEYS = sorted(TRAIT_CONFIG.keys())
TRAIT_SUMMARY = 'traits-none' if not TRAIT_KEYS else 'traits-' + '_'.join(_slugify_filename(name) for name in TRAIT_KEYS)
STEER_SUMMARY = f"trait-mix-{_slugify_filename(STEERING_CONFIG_FILE.stem, default='steer')}"
METADATA_PARTS = [
    STEER_SUMMARY,
    TRAIT_SUMMARY,
    f"temp-{_format_numeric_for_filename(SIM_STEER_TEMPERATURE)}",
    f"k-{SIM_SAMPLES}",
    f"turns-{SIM_EXCHANGES}",
    f"mix-{'on' if SIM_MIX_TRAITS else 'off'}"
]

print(f'Loaded simulation: {SIMULATION_CONFIG_FILE} | steering: {STEERING_CONFIG_FILE} | tasks: {STEER_TASKS or "<none>"}')


Loaded simulation: configs/simulation_config.json | steering: configs/steering_config_airline.json | tasks: ['airline support']


In [65]:
import os

if CLIENT_ASSISTANT_MODEL_API_KEY:
    os.environ['OPENAI_API_KEY'] = CLIENT_ASSISTANT_MODEL_API_KEY
if CLIENT_ASSISTANT_MODEL_URL:
    os.environ['OPENAI_BASE_URL'] = CLIENT_ASSISTANT_MODEL_URL
if CLIENT_STEER_API_KEY:
    os.environ['STEER_API_KEY'] = CLIENT_STEER_API_KEY


## Client setup

Initializes the Collinear client and applies optional custom system prompts.


In [54]:
# Client setup
if not CLIENT_ASSISTANT_MODEL_API_KEY:
    raise RuntimeError('assistant_model_api_key must be set in configs/simulation_config.json')

client = Client(
    assistant_model_url=CLIENT_ASSISTANT_MODEL_URL,
    assistant_model_api_key=CLIENT_ASSISTANT_MODEL_API_KEY,
    assistant_model_name=CLIENT_ASSISTANT_MODEL_NAME,
    steer_api_key=CLIENT_STEER_API_KEY,
    timeout=CLIENT_TIMEOUT,
    max_retries=CLIENT_MAX_RETRIES,
    rate_limit_retries=CLIENT_RATE_LIMIT_RETRIES,
)

runner = client.simulation_runner


## Generate simulated user interactions

Runs simulations and writes a JSONL dataset with conversation, assistant response, and steering persona.


In [67]:
# Generate simulations

simulations = client.simulate(
    steer_config=STEER_CONFIG,
    k=SIM_SAMPLES,
    num_exchanges=SIM_EXCHANGES,
    batch_delay=SIM_DELAY,
    steer_temperature=SIM_STEER_TEMPERATURE,
    steer_max_tokens=SIM_STEER_MAX_TOKENS,
    mix_traits=SIM_MIX_TRAITS,
    max_concurrency=SIM_MAX_CONCURRENCY,
)

results_dir = Path(RESULTS_DIR)
results_dir.mkdir(parents=True, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = '_'.join(part for part in METADATA_PARTS if part) + f"_{timestamp}.jsonl"
dataset_path = results_dir / filename

with dataset_path.open('w', encoding='utf-8') as dataset_file:
    for simulation in simulations:
        dataset_row = make_dataset_row(runner, simulation)
        dataset_file.write(json.dumps(dataset_row, ensure_ascii=False) + '\n')
print(f'Wrote dataset to: {dataset_path}')

with open(dataset_path) as f:
    for i, x in enumerate(f):
        print("="*8)
        print(f"Conversation {i+1}")
        print("="*8)
        pprint(json.loads(x)["steering_persona"])
        print(json.loads(x)["conversation"])
        if json.loads(x)["last_response"] ==  "###STOP###":
            print("user: ", json.loads(x)["last_response"])
        else:
            print("assistant: ", json.loads(x)["last_response"])
        print()


User/Assistant turns:  86%|████████▌ | 86/100 [01:09<00:11,  1.23query/s]

Wrote dataset to: results/trait-mix-steering_config_airline_traits-confusion_impatience_skeptical_temp-0p7_k-10_turns-10_mix-off_20250924_155634.jsonl
Conversation 1
{'characteristics': {'age': 16,
                     'gender': 'male',
                     'intent': 'Rebook Due to Disruption',
                     'language': 'English',
                     'location': 'USA',
                     'occupation': 'student',
                     'task': 'airline support'},
 'traits': {'impatience': -2}}
user: My flight from Chicago to Miami got delayed due to thunderstorms and I'm supposed to be in Miami for a school trip, can you help me rebook for today?
assistant: I’m sorry to hear about the delay. Let me check the next available flights for you. I’ll need your confirmation number or the email you used when booking, and if you have any preferred times or seating preferences. Once I have those details, I can pull up the options for today and secure a seat for you on a flight that works 

## Upload simulations as dataset and load judge model on Together

Uploads the dataset to Together and starts a safety-score evaluation.


In [ ]:
together_client = together.Together(api_key=CLIENT_ASSISTANT_MODEL_API_KEY)

# Upload dataset
uploaded_file = together_client.files.upload(file=str(dataset_path), purpose=TOGETHER_UPLOAD_PURPOSE)
uploaded_file_id = uploaded_file.id if hasattr(uploaded_file, 'id') else uploaded_file['id']

# Create evaluation
evaluation_requestor = api_requestor.APIRequestor(client=together_client.client)
evaluation_payload = {
    'type': TOGETHER_EVAL_TYPE,
    'parameters': {
        'judge': {
            'model': ASSESS_JUDGE_MODEL_NAME,
            'model_source': TOGETHER_JUDGE_MODEL_SOURCE,
            'system_template': JUDGE_SYSTEM_PROMPT,
        },
        'input_data_file_path': uploaded_file_id,
        'model_to_evaluate': TOGETHER_MODEL_TO_EVALUATE,
        'min_score': TOGETHER_MIN_SCORE,
        'max_score': TOGETHER_MAX_SCORE,
        'pass_threshold': TOGETHER_PASS_THRESHOLD,
    },
}
evaluation_create_response, _, _ = evaluation_requestor.request(
    options=TogetherRequest(method='POST', url='evaluation', params=evaluation_payload),
    stream=False,
)
evaluation_data = getattr(evaluation_create_response, 'data', evaluation_create_response)
workflow_id = evaluation_data.workflow_id if hasattr(evaluation_data, 'workflow_id') else evaluation_data['workflow_id']
status_text = str(getattr(evaluation_data, 'status', 'pending')).lower()
print(f'Started evaluation: {workflow_id} (status={status_text})')

## Eval results and analysis

Polls for completion, enriches results with personas, and prints a summary.


In [ ]:
# Poll Together until complete. Download and print results

deadline = time.time() + TOGETHER_POLL_TIMEOUT_SECONDS
while time.time() < deadline:
    evaluation_status = together_client.evaluation.status(workflow_id)
    status_text = str(getattr(evaluation_status, 'status', 'pending')).lower()
    print(f'Status: {status_text}')
    if status_text.endswith(('completed', 'success', 'failed', 'error', 'user_error')):
        results_obj = getattr(evaluation_status, 'results', None)
        print(results_obj)
        if isinstance(results_obj, dict) and results_obj.get('result_file_id'):
            results_path = dataset_path.parent / f"{RESULTS_FILENAME_PREFIX}{workflow_id}_results.jsonl"
            together_client.files.retrieve_content(results_obj['result_file_id'], output=str(results_path))
            print(f'Downloaded results to: {results_path}')

            # Enrich results in place with steering_persona from dataset
            try:
                persona_lookup = load_persona_lookup(dataset_path)
                enrich_results_inplace(results_path, persona_lookup)
                print('Enriched results with steering_persona.')
            except Exception as error:
                print(f'Note: could not enrich results with steering_persona: {error}')

            _summarize_results(results_path)
        break
    time.sleep(TOGETHER_POLL_INTERVAL_SECONDS)
else:
    print('Timed out waiting for evaluation to complete.')
